In [1]:
from tcl_client import *
cl = Tcl_client()
cl.open()

In [2]:
cl.send_command("target 1")
cl.send_command("jtag target 2")
cl.send_command("jcm_lite::update_active_device")

'EOF'

In [3]:
cl.send_command("fpga -f ../../dut/outputs/kronos.bit")

'EOF'

In [4]:
import serial
BAUD = 115200
DEV = "COM4"

In [5]:
new_serial = serial.Serial(port=DEV, baudrate=BAUD,timeout=5,)

In [6]:
new_serial.reset_input_buffer()
print(new_serial.readline())

b'00:38A7\n'


In [7]:
new_serial.reset_input_buffer()
cl.send_command("fpga -f ../../tmr/outputs/kronos_tmr.bit")
print(new_serial.readline())

b'00:38A7\n'


In [8]:
cl.send_command("jcm_lite::inject_random_fault").split()

['00020814', '77', '16', '1', 'EOF']

In [9]:
import re
from collections import Counter
num_procs = 1
TELEMETRY_KEY = "([0-9A-F]{2}):"
for x in range(num_procs):
    TELEMETRY_KEY += "([0-9A-F]{4})"
    


In [10]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [11]:
new_serial.reset_input_buffer()
for y in range(10):
    cl.send_command("fpga -f ../../dut/outputs/kronos.bit")
    new_serial.reset_input_buffer()
    old_value = -1
    injection = ['0','0','0','0']
    for x in range(100000):
        printProgressBar(x,100000)
        serial_string = str(new_serial.readline())
        if old_value != -1 and injection[3] == '1':
            cl.send_command("jcm_lite::inject_fault {} {} {}".format(injection[0],injection[1],injection[2]))
        m = re.search(TELEMETRY_KEY,serial_string)
        if m:
            if old_value == -1:
                old_value = int(m.group(2),16)
            else:
                current_value = int(m.group(2),16)
                if current_value < 60 and old_value > 0xFFD0:
                    current_value += 0x10000
                diff = current_value - old_value
                if diff == 39 or diff == 40:
                    pass
                else:
                    # print (m.group(2))
                    print ("Failure: ",y,x,m.group(1),m.group(2),diff,injection)
                    break
                old_value =  int(m.group(2),16)
        else:
            print ("Failure: ",y,x,serial_string,injection)
            break
            

        cl.send_command("jcm_lite::ctl0_nmask")
        injection = cl.send_command("jcm_lite::inject_random_fault").split()

Failure:  0 130 82 4CB8 10 ['00400D09', '81', '3', '1', 'EOF']----------------------------------------| 0.1% 
Failure:  1 65 41 42A2 11 ['00400D20', '62', '10', '1', 'EOF']----------------------------------------| 0.1% 
Failure:  2 35 23 3DFA 12 ['00400F8E', '72', '24', '1', 'EOF']----------------------------------------| 0.0% 
Failure:  3 34 22 3DD0 11 ['00400E0D', '68', '18', '1', 'EOF']----------------------------------------| 0.0% 
Failure:  4 55 37 4113 10 ['00400E94', '66', '6', '1', 'EOF']-----------------------------------------| 0.1% 
Failure:  5 58 3A A2BC 24892 ['00400C17', '79', '1', '1', 'EOF']--------------------------------------| 0.1% 
Failure:  6 65 41 42A1 10 ['00400F15', '78', '18', '1', 'EOF']----------------------------------------| 0.1% 
Failure:  7 26 1A 3C94 11 ['00400982', '89', '14', '1', 'EOF']----------------------------------------| 0.0% 
Failure:  8 366 6E 737B 554 ['00400D16', '45', '2', '1', 'EOF']---------------------------------------| 0.4% 
Failure:  

In [12]:
new_serial.reset_input_buffer()
for y in range(10):
    cl.send_command("fpga -f ../../tmr/outputs/kronos_tmr.bit")
    new_serial.reset_input_buffer()
    old_value = -1
    injection = ['0','0','0','0']
    for x in range(100000):
        printProgressBar(x,100000)
        serial_string = str(new_serial.readline())
        if old_value != -1 and injection[3] == '1':
            cl.send_command("jcm_lite::inject_fault {} {} {}".format(injection[0],injection[1],injection[2]))
        m = re.search(TELEMETRY_KEY,serial_string)
        if m:
            if old_value == -1:
                old_value = int(m.group(2),16)
            else:
                current_value = int(m.group(2),16)
                if current_value < 60 and old_value > 0xFFD0:
                    current_value += 0x10000
                diff = current_value - old_value
                if diff == 39 or diff == 40:
                    pass
                else:
                    # print (m.group(2))
                    print ("Failure: ",y,x,m.group(1),m.group(2),diff,injection)
                    break
                old_value =  int(m.group(2),16)
        else:
            print ("Failure: ",y,x,serial_string,injection)
            break
            

        cl.send_command("jcm_lite::ctl0_nmask")
        injection = cl.send_command("jcm_lite::inject_random_fault").split()

Failure:  0 6351 CF 38A4 9783 ['00400B8C', '65', '10', '1', 'EOF']------------------------------------| 6.4% 
Failure:  1 2251 CB 0F1A -34519 ['00400E00', '95', '22', '1', 'EOF']----------------------------------| 2.3% 
Failure:  2 5228 6C 6423 12 ['00400F8F', '32', '19', '1', 'EOF']--------------------------------------| 5.2% 
Failure:  3 6309 b'\r' ['00000007', '98', '7', '1', 'EOF']--------------------------------------------| 6.3% 
Failure:  4 1699 A3 404A 10 ['00400D1B', '84', '28', '1', 'EOF']--------------------------------------| 1.7% 
Failure:  5 4729 79 16CB 43 ['00400C9B', '95', '22', '1', 'EOF']--------------------------------------| 4.7% 
Failure:  6 8091 9B 389A 6159 ['00400307', '68', '14', '1', 'EOF']------------------------------------| 8.1% 
Failure:  7 2435 b'\r' ['0000159D', '54', '1', '1', 'EOF']--------------------------------------------| 2.4% 
Failure:  8 6192 30 F9CA 12 ['00400302', '67', '18', '1', 'EOF']--------------------------------------| 6.2% 
Failure:  